In [ ]:
from functools import partial

import adaptive
import adaptive_scheduler


def h(x, width=0.01, offset=0):
    import numpy as np
    import random

    for _ in range(10):  # Burn some CPU time just because
        np.linalg.eig(np.random.rand(1000, 1000))

    a = width
    return x + a ** 2 / (a ** 2 + (x - offset) ** 2)


offsets = [i / 10 - 0.5 for i in range(5)]

combos = adaptive.utils.named_product(offset=offsets, width=[0.01, 0.05])

learners = []
fnames = []

for combo in combos:
    f = partial(h, **combo)
    learner = adaptive.Learner1D(f, bounds=(-1, 1))
    fname = adaptive_scheduler.utils.combo2fname(combo, folder="data")
    fnames.append(fname)
    learners.append(learner)
name = "example"
scheduler = adaptive_scheduler.scheduler.SLURM(
    cores_per_node=tuple(range(1, 11)),
    nodes=(1,)*10,
    partition="local-all",
    executor_type="process-pool",
    log_folder="logs",
    exclusive=False,
)
run_manager = adaptive_scheduler.RunManager(
    learners=learners,
    fnames=fnames,
    scheduler=scheduler,
    goal=0.01,
    job_name=f"{name}",
    max_fails_per_job=5,
    max_simultaneous_jobs=50,
    db_fname=f"{name}-database.json",
    log_interval=30,
    save_interval=30,
    save_dataframe=True,
    cleanup_first=False,
)
run_manager.start()